In [61]:
from Grafo import *
import numpy as np
from operator import index
from traceback import print_tb
# import o alfabeto em codigo Ascii 
import string 
# manipular os dados de excel com xlrd
import xlrd

### Criacao do Alfabeto


In [62]:
# VARAIBLES GLOBALES
tamanhoAlfabeto = len(string.printable)
alfabeto = []
# Ajuste do numero de estados para criar o automato
numEstados = 29 #son 28 estados en total pero como el rango de [0;28] tiene que se x<41
estadosList = [i for i in range(numEstados)]

def criaAlfabeto():
    alfabeto.extend(string.printable)  #cria uma nova lista contendo o alfabeto

estadosFinais=[1,3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,21,22,23,24,25,28]

### Leitura e criacao da Tabela de Simbolos

In [63]:
def criaTabelaSimbolos(tabelaSimbolos):
    try:
        # import banco de dados palabras reservadas (excel.xls) 
        #excel = xlrd.open_workbook('C:/Users/gaboh/Documents/GitHub/BNF/Tabela-Simbolos2.xls') #import xlrd
        excel = xlrd.open_workbook('/home/moskbr/Documentos/git/BNF/Tabela-Simbolos_3.xls')
        sh = excel.sheet_by_index(0) #Folha 1 ou sheet 1
    except:
        print("Exception ->Tabela de simbolos tem que ser um doc Excel do tipo .xls")

    
    numColumnas = sh.nrows-1
    for key in range(numColumnas):
        cellCadeia = sh.cell(key+1,1).value
        cellCategoria = sh.cell(key+1,2).value
        cadeia = str(cellCadeia)
        tabelaSimbolos[cadeia.upper()] = cellCategoria
    # Tabela Simbolos -> colocar as pabras dentro de uma tabela hash ( Em python o dictionary representa a tabela hash )
    #print(tabelaSimbolos)

### Construcao do Automato 

In [64]:
def criaAutomato(grafo): 
    
    for x in range(numEstados): agregar(grafo, estadosList[x]) # Cria o grafo de 40 estados
    letras=[]
    algarismos=[]

    for cont in range(0,52): #armazena so as letras do alfabeto
        letras.append(alfabeto[cont+10])
    for cont in range(0,10): # armazena so os numeros do alfabeto 
        algarismos.append(alfabeto[cont])
    
    relacionar(grafo, estadosList[0], estadosList[1], algarismos)
    relacionar(grafo, estadosList[1], estadosList[1], algarismos)
    relacionar(grafo, estadosList[2], estadosList[3], algarismos)
    relacionar(grafo, estadosList[3], estadosList[3], algarismos)
    relacionar(grafo, estadosList[0], estadosList[4], letras)
    relacionar(grafo, estadosList[4], estadosList[4], alfabeto)
    relacionar(grafo, estadosList[1], estadosList[2], '.')
    relacionar(grafo, estadosList[0], estadosList[5], '\\')
    relacionar(grafo, estadosList[5], estadosList[6], '\\')
    relacionar(grafo, estadosList[0], estadosList[7], '(')
    relacionar(grafo, estadosList[0], estadosList[8], ')')
    relacionar(grafo, estadosList[0], estadosList[9], '{')
    relacionar(grafo, estadosList[0], estadosList[10], '}')
    relacionar(grafo, estadosList[0], estadosList[11], '=')
    relacionar(grafo, estadosList[11], estadosList[12], '=')
    relacionar(grafo, estadosList[0], estadosList[13], '+')
    relacionar(grafo, estadosList[13], estadosList[14], '+')
    relacionar(grafo, estadosList[0], estadosList[15], '-')
    relacionar(grafo, estadosList[15], estadosList[16], '-')
    relacionar(grafo, estadosList[0], estadosList[17], '*')
    relacionar(grafo, estadosList[0], estadosList[18], '/')
    relacionar(grafo, estadosList[0], estadosList[19], '%')
    relacionar(grafo, estadosList[0], estadosList[20], '!')
    relacionar(grafo, estadosList[20], estadosList[21], '=')
    relacionar(grafo, estadosList[0], estadosList[22], '>')
    relacionar(grafo, estadosList[22], estadosList[23], '=')
    relacionar(grafo, estadosList[0], estadosList[24], '<')
    relacionar(grafo, estadosList[24], estadosList[25], '=')
    relacionar(grafo, estadosList[0], estadosList[26], '"')
    relacionar(grafo, estadosList[26], estadosList[27], algarismos)
    relacionar(grafo, estadosList[26], estadosList[27], letras)
    relacionar(grafo, estadosList[27], estadosList[28], '"')


### Análise léxica (Scanner)
* Tratamento de Espacios 
* Eliminar comentários
* Contar número de linhas

In [65]:
def divideLexemas(lexema, contColumnas, tabelaSimbolos):
    for palavra in lexema:
        for letra in palavra:
            #contColumnas+=1

            if letra == "\"":# encontra abre aspas
                fecha_aspas = palavra[palavra.index(letra)+1]
                i = 1
                while fecha_aspas != "\"":# procura fecha aspas
                    if i < len(palavra):
                        fecha_aspas = palavra[palavra.index(letra)+i]
                        i += 1
                if i == len(palavra) and fecha_aspas == "\"": break

            try:
                if (palavra.index(letra) != len(palavra)):
                    prox = palavra[palavra.index(letra)+1]
            except:
                prox = None

            if prox in tabelaSimbolos:
                letra = letra + prox
                        
            if letra in tabelaSimbolos and len(palavra) > len(letra):
                lista = palavra.split(letra)
                lista.append(letra)
                      
                for s in lista:
                    if s != '':
                        lexema.append(s)
                                
                lista.clear()
                lexema.remove(palavra)
                break
    
    for palavra in lexema: # ajusta o ponto e virgula para o final da lista
        if palavra == ";":
            lexema.append(palavra)
            lexema.remove(palavra)
            break

    return lexema

# essa função elimina os espaços dentro de uma cadeia estre as aspas
def entreAspas(linha, contLinhas):
    indice_aspas = linha.index("\"")
    i = indice_aspas+1
    entre_aspas = "\""
    while linha[i] != "\"":# separa a cadeia que esta entre as aspas
      entre_aspas = entre_aspas + linha[i]
      i += 1
      if i >= len(linha):
        print("Erro na linha {}: Esperado caractere Fecha Aspas", contLinhas)
    entre_aspas = entre_aspas + "\""
    entre_aspas = entre_aspas.replace(" ", '')# elimina os espaços
    linha = linha[0:indice_aspas: ] + entre_aspas + linha[i+1: :]# devolve para a lista original
    return linha

### Recuperacao de Erros

In [66]:
def Erro(tipo, X, Y, erro):
    if tipo == "Lexico":
        print("[Erro: Análise {}]: símbolo incorreto na linha {}, coluna {}. O símbolo {} não pertence ao alfabeto da linguagem.".format(tipo, X, Y, erro))
    
    #print("[Erro: Análise Léxica]: símbolo incorreto na linha X, coluna Y. O variavel “@” não pertence ao alfabeto da linguagem.")



### CYK CODE & BNF

In [67]:
def analiseSintatica(tokens_lexema, tabelaSimbolos):
    if (tokens_lexema[0]in tabelaSimbolos.values()):
        if( tokens_lexema[0] == 'TK_While' and 'TK_If'):
            #simbolos no terminais
            no_terminales_0 = []
            terminales_0 = []

            #regras da gramatica 
            Regras_0 = {
                
            }
        elif(tokens_lexema[0] == 'TK_Inteiro' and 'TK_Flutuante' and 'TK_Char'):
            #simbolos no terminais
            no_terminales_1 = []
            terminales_1 = []

            #regras da gramatica 
            Regras_1 = {
                
            }
        elif(tokens_lexema[0] == 'TK_Write'):
            #simbolos no terminais
            no_terminales_2 = []
            terminales_2 = []

            #regras da gramatica 
            Regras_2 = {
                
            }
        elif(tokens_lexema[0] == 'TK_Read'):
            #simbolos no terminais
            no_terminales_3 = []
            terminales_3 = []

            #regras da gramatica 
            Regras_3 = {
                
            }
        elif(tokens_lexema[0] == 'TK_Comentario'):
            #simbolos no terminais
            no_terminales_4 = []
            terminales_4 = []

            #regras da gramatica 
            Regras_4 = {
                
            }                        
        elif(tokens_lexema[0] == 'TK_Identificador'):
            #simbolos no terminais
            no_terminales_5 = []
            terminales_5 = []

            #regras da gramatica 
            Regras_5 = {
                
            }

        elif(tokens_lexema[0] == 'TK_Fecha_Chaves'):
            #simbolos no terminais
            no_terminales_6 = []
            terminales_6 = []

            #regras da gramatica 
            Regras_6 = {
                
            }    
        else:
            print("erro semantica, primeira palavra")      

   

In [68]:
#Funcao do algoritmo CYK
# w eh a lista que ele debe fazer a leitura e identificacao 
def cykParser(w, terminais, noterminais, regra ):
    n = len(w) 

## Main e leitura do arquivo

In [69]:
def main():
    criaAlfabeto()
    tabelaSimbolos = {} #dictionary
    criaTabelaSimbolos(tabelaSimbolos)

    tokens_lexema = []

    grafo = Grafo()
    criaAutomato(grafo)
    # variaveis para armazenar os tokens dependo do tipo 
    contFloat = 0 
    contInt = 0
    contString = 0 
    contEntreAspas = 0
    
    # Analise Lexica
    contLinhas = 0
    contColumnas = 0
    lista = []
    # Leitura do arquivo 
    with open('teste.teste') as arquivo:
        for linha in arquivo:
            if linha == '\n': continue # desconsidera linhas vazias
            contLinhas += 1

            if "\\" in linha:
                linha = linha[0:linha.index("\\")]

            if "\"" in linha:# a função elimina os espaços dentro da cadeia que esta estre aspas antes do
                linha = entreAspas(linha, contLinhas)# split abaixo para que a cadeia nao fique fragmentada
            
            lexema = linha.split()
            lista = divideLexemas(lexema, contColumnas, tabelaSimbolos)
            
            for w in lista:
                contColumnas+=1
                if(verificaPalavra(grafo, w, estadosFinais)):
                    try: 
                        tokens_lexema.append(tabelaSimbolos[w.upper()]) 
                        #print(tabelaSimbolos[w.upper()])
                    except: 
                        if(w.isdigit()):
                            tabelaSimbolos[w] = "TK_Inteiro_"+str(contInt)
                            tokens_lexema.append("TK_Inteiro") 
                            contInt+=1
                        elif(isfloat(w)): 
                            tabelaSimbolos[w] = "TK_Flutuante_"+str(contFloat)
                            tokens_lexema.append("TK_Flutuante") 
                            contFloat+=1
                        elif w[0] == "\"":
                            tabelaSimbolos[w] = "TK_Entre_Aspas_"+str(contEntreAspas)
                            tokens_lexema.append("TK_Entre_Aspas")
                            contEntreAspas += 1
                        else:
                            tabelaSimbolos[w.upper()] = "TK_Identificador_"+str(contString)
                            tokens_lexema.append("TK_Identificador") 
                            contString+=1
                else :
                    print("Erro Automato:") 
                    #tokens_lexema.clear()
                    Erro("Lexico", contLinhas, contColumnas, w)
                    break
                
            print(tokens_lexema)
            #analiseSintatica(tokens_lexema, tabelaSimbolos)
            tokens_lexema.clear()

    print(tabelaSimbolos)
    print("Numero de linhas do Codigo: {}".format(contLinhas))
    
    


In [70]:
if __name__ == "__main__":
    main()

['TK_Flutuante']
{'WHILE': 'TK_While', 'IF': 'TK_If', 'INT': 'TK_Int', 'FLOAT': 'TK_Float', 'STRING': 'TK_String', 'WRITE': 'TK_Write', 'READ': 'TK_Read', '(': 'TK_Abre_Parenteses', ')': 'TK_Fecha_Parenteses', '{': 'TK_Abre_Chaves', '}': 'TK_Fecha_Chaves', '=': 'TK_Atribui_Valor', '+': 'TK_Soma', '-': 'TK_Subtracao', '*': 'TK_Multiplicacao', '/': 'TK_Divisao', '%': 'TK_Mod', '++': 'TK_Incrementa_Um', '--': 'TK_Decrementa_Um', '==': 'TK_Verifica_Igual', '!=': 'TK_Verifica_Diferente', '>': 'TK_Maior', '<': 'TK_Menor', '>=': 'TK_Maior_Igual', '<=': 'TK_Menor_Igual', '\\\\': 'TK_Comentario', '123.233': 'TK_Flutuante_0'}
Numero de linhas do Codigo: 1
